# Measure Trajectory Curvature

Update Matan 20240202

Written Matan 2019 05 11

This scrit measures the curvature of a trajectory over a window $\tau$.
Works well with trajectories that are rather persistent.
Was not tested with very noisy trajectories.

## Curvature measurement

The radius of curvature between steps 1, 2 and 3 is:

$$ R = \frac{\left|a-b\right|}{2 sin\theta}$$

where $a \equiv r_3-r_1$, and $b\equiv r_3-r_2$, and $\theta$ is the angle between $a$ and $b$.
 
Then we need to compute the radius of curvature for different time intervals, $r\left(t\right)$, $r\left(t+\tau\right)$, $r\left(t+2\tau\right)$: 

$$ \left<R\left(\tau\right)\right>_t = \left<\frac{a\left(t;\tau\right)-b\left(t;\tau\right)}{2 sin\theta\left(t;\tau\right)}\right>_t$$

Alternatively, this can also be parametrized on the curve-length.

###
$$ \left<\vec{V}\left(t\right)\cdot\vec{V}\left(t+\tau\right)\right>_t $$

In [46]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import time
import numpy as np
from scipy import interpolate

%matplotlib inline
def l2n(x): 
    return np.asarray(x)


#mpl.rc('text',usetex=True)
plt.style.use('dark_background')

params = {'legend.fontsize': 22,
          'figure.figsize': (8, 6),
         'axes.labelsize': 24,
         'axes.titlesize':24,
         'xtick.labelsize':22,
         'ytick.labelsize':22}
plt.rcParams.update(params)
plt.rc('xtick',labelsize=22)
plt.rc('ytick',labelsize=22)

In [47]:
#class calcPersistence:
#    

#@njit    
def _autoCorr(v,maxLag):
    vac = np.zeros(maxLag) #initialize the correlation function
    vMag = np.sum(np.sqrt(v*v),axis=0) #velocity magnitudes
    
    vac[0] = np.mean( np.sum( v*v ,axis=0)/(vMag*vMag));

    for tau in range(1,maxLag):            
        vac[tau] = np.mean( np.sum( v[:,tau:]*v[:,:-tau]/(vMag[tau:]*vMag[:-tau]) ,axis=0));
  
    return vac

def velocityTimeAutoCorrelation(t,r,maxLag=10):
    #Return the autocorelation of an instanteneous veolocity 
    #t - time, x - a position vector
    maxLag = min(maxLag,len(t)-1)

    tt = np.zeros(r.shape[0])+t[:,None]
    tt = tt.T
    v = np.diff(r)/np.diff(tt)

    vac = _autoCorr(v,maxLag)
    return vac

def calcInstantaneousCurvature(r):
    
    #calculates the local curvature at every point
    #r is a vector array of location
    
    #calculate difference between positions 3&1 and 2&1
    a = (np.roll(r,-2)-r)[:,:-2]
    b = (np.roll(r,-1)-r)[:,:-2]
    aNorm = np.linalg.norm(a)
    bNorm = np.linalg.norm(b)
    sinTheta = np.cross(a,b,axis=0)/(aNorm*bNorm)

    R = (a-b)/(2*sinTheta)

    return R


def calcWindowVectorCurvature(r,halfWindow=1):
    
    window=halfWindow*2
    
    #calculates the local curvature at every point
    #r is a vector array of location
    
    #calculate difference between positions 3&1 and 2&1
    a = (np.roll(r,-window)-r)[:,:-window]
    b = (np.roll(r,-halfWindow)-r)[:,:-window]
    aNorm = np.linalg.norm(a,axis=0)
    bNorm = np.linalg.norm(b,axis=0)
    sinTheta = np.cross(a,b,axis=0)/(aNorm*bNorm)

    R = (a-b)/(2*sinTheta)

    return R

def calcWindowScalarCurvature(r,halfWindow=1):
    
    window=halfWindow*2
    
    #calculates the local curvature at every point
    #r is a vector array of location
    
    #calculate difference between positions 3&1 and 2&1
    a = (np.roll(r,-window)-r)[:,:-window]
    b = (np.roll(r,-halfWindow)-r)[:,:-window]
    aNorm = np.linalg.norm(a,axis=0)
    bNorm = np.linalg.norm(b,axis=0)
    sinTheta = np.cross(a,b,axis=0)/(aNorm*bNorm)

    R = np.linalg.norm(a-b,axis=0)/(2*sinTheta)

    return R

def calcWindowTangent(r,window=1):
    #computer the local tangent.
    
    #calculates the local curvature at every point
    #r is a vector array of location
    
    #calculate difference between positions 3&1 and 2&1
    a = (np.roll(r,-window)-r)[:,:-window]
    
    
    return a

# Load Data 

In [53]:
folder_name='D:\\Eden\\new_exp_test_15_11_23\\5_2_24'
linking_file_name='C0659.MP4_locatedFull20240205_bot_Eden.csv_linked.csv'
linkedDataFileName = '{}\\{}'.format(folder_name,linking_file_name)
tl = pd.read_csv(linkedDataFileName)
tl.head()
#np.roll(z,-2)-z

,frame,Unnamed: 0,frame.1,x,y,r,particle
0,3772,1783,3772,806.5,217.5,31.6,2
1,3776,1784,3776,826.5,224.5,37.2,2
2,3780,1785,3780,826.5,230.5,37.2,2
3,3782,1786,3782,818.5,229.5,32.7,2
4,3784,1787,3784,813.5,230.5,30.5,2


# Calc Curvature 

In [50]:
halfWindow = 20
window=halfWindow*2
x1 = tl[tl.particle==0].x
y1 = tl[tl.particle==0].y

x2 = tl[tl.particle==1].x
y2 = tl[tl.particle==1].y

N1 = len(x1)
N2 = len(x2)

r1 = np.concatenate((x1,y1)).reshape(2,N1)
r2 = np.concatenate((x2,y2)).reshape(2,N2)

R1 = calcWindowScalarCurvature(r1,halfWindow)
R2 = calcWindowScalarCurvature(r2,halfWindow)
print(R1)




[130.10092103 131.13307832 127.69232378 ... 198.95465328 181.37450251
 170.65364083]


C:\Users\User\AppData\Local\Temp\ipykernel_106368\3162245244.py:77: RuntimeWarning: divide by zero encountered in divide
  R = np.linalg.norm(a-b,axis=0)/(2*sinTheta)


In [51]:
arenaSizePixelsx = 955 # [pixels]
arenaSizePixelsy=470
arenaSizeCm = 100# [cm]
#account for scalebar
ppcx = arenaSizePixelsx/arenaSizeCm #pixels per centimeter
ppcy=arenaSizePixelsy/arenaSizeCm
X1 = r1[0,halfWindow*2:]/ppcx
Y1 = r1[1,halfWindow*2:]/ppcy
X2 = r2[0,halfWindow*2:]/ppcx
Y2 = r2[1,halfWindow*2:]/ppcy

t1 = np.arange(len(X1))
t2 = np.arange(len(X2))

t1New = t1[::2]
t2New = t2[::2]

R1new = interpolate.interp1d(t1,R1)(t1New)
#R2new = interpolate.interp1d(t2,R2)(t2New)

In [ ]:
print(np.median(R1new)/ppc)
#print(np.median(R2new)/ppc)

# Plot All Together 

In [52]:

fig,ax = plt.subplots(1,2,figsize=(12,6))
#fig,ax = plt.subplots(1,1)

ax[0].scatter(X1,Y1,c=np.roll(1/np.abs(R1),-halfWindow),cmap='Blues',s=ppc*6,alpha=1)
ax[0].scatter(X2,Y2,c=np.roll(1/np.abs(R2),-halfWindow),cmap='Oranges',s=ppc*6,alpha=0.8)

ax[0].plot(X1,Y1,c='k',linewidth=2)#0.5)
ax[0].plot(X2,Y2,c='k',linewidth=2)#0.5)
ax[0].axis([0,100,0,100])

ax[1].plot(t1New,R1new/ppc,c='LightBlue',linewidth=3,alpha=0.9)
ax[1].plot(t2New,R2new/ppc,c='Orange',linewidth=3,alpha=0.9)
#=ax[1].axis([160,330,-200,200])
ax[1].grid(alpha=0.3)


ax[0].set_xlabel('[cm]')
ax[0].grid(alpha=0.2)
ax[0].set_ylabel('[cm]')

ax[1].set_xlabel('[sec]')
ax[1].set_ylabel('Curvature [cm]')
ax[1].yaxis.set_label_position("right")
ax[1].yaxis.tick_right()
ax[0].text(62,5,'R = 11 cm',fontsize=24,color='LightBlue')
ax[0].text(62,60,'R = -44 cm',fontsize=24,color='Orange')
fig.savefig('{}_curvature_figure_{}.png'.format(folder_name,linking_file_name))

NameError: name 'R2new' is not defined

Error in callback <function _draw_all_if_interactive at 0x00000184F03945E0> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1200x600 with 2 Axes>

In [42]:
import trackpy as tp

In [43]:
tp.plot_traj(tl)

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 800x600 with 1 Axes>

<AxesSubplot: xlabel='x [px]', ylabel='y [px]'>

In [45]:
print(np.median(R1new)/ppc)
#print(np.median(R2new)/ppc)

5.386004256541343
